In [14]:
import pandas as pd
import numpy as np

In [15]:
#Returns consolidated df with coincident rows rolled up. Coincident refers to rows
#having same date/time that are populated in various columns.  In case of rows where same
#column is populated as previous, coincident row, no consolidation occurs unless
#column is in a separate, 'override' list.
#Version of 3/16/20 - uses .loc instead of .iloc; consolidates "downward"
#Version of 4/11/20 - fixed issue with non-consecutive index.  Now flexible to whatever index because of
#                    df.index.get_loc(idx) to convert everything to .iloc/row sequence basis

def RollupCoincidentRows(df, dt_col, lst_cols, lst_override, IsFlagConflicts, IsDeleteCoinc):
    df = df.copy()
    
    #Add flag columns and populate with defaults
    kp_col, confl_col, coinc_col = 'keep', 'RowConflict', 'IsCoincident'
    df.loc[:,kp_col], df.loc[:,confl_col], df.loc[:, coinc_col] = True, False, False

    #Record column indices for lst_cols, kp_col, confl_col and coinc_col
    lst_col_indices = []
    for col in lst_cols:
        lst_col_indices.append(df.columns.get_loc(col))
    j_kp_col = df.columns.get_loc(kp_col)
    j_confl_col = df.columns.get_loc(confl_col)
    j_coinc_col = df.columns.get_loc(coinc_col)
    j_dt_col = df.columns.get_loc(dt_col)

    for idx, row in df.iterrows():
        i = df.index.get_loc(idx)
        if i == 0: continue

        #Skip rows already flagged for deletion
        iPrev = i - 1
        while iPrev > 0 and not df.iloc[iPrev,j_kp_col]:
            iPrev = iPrev - 1

        #Consolidate if i and iPrev are coincident and i's data don't conflict
        if row[dt_col] == df.iloc[iPrev, j_dt_col]:
            df.iloc[i, j_coinc_col], df.iloc[iPrev, j_coinc_col] = True, True

            #Default is no conflicts; keep=False for row i
            IsIrresolvable, IsConflict = False, False
            df.iloc[i, j_kp_col] = False

            #Check each column
            for col, j in zip(lst_cols,lst_col_indices):
                if not IsRowConflict(df, i,iPrev, col):
                    if IsNullCell(df, iPrev, col): df.iloc[iPrev,j] = row[col]
                elif col in lst_override:
                    df.iloc[iPrev,j] = row[col]
                    IsConflict = True
                else: IsConflict, IsIrresolvable = True, True

                #Flag conflict whether overridden or not
                if IsConflict and IsFlagConflicts:
                    df.iloc[iPrev, j_coinc_col], df.iloc[i, j_confl_col] = True, True

            #Don't drop the row if unresolved conflicts
            if IsIrresolvable: df.iloc[i, j_kp_col] = True

    #Return after dropping flagged rows and Boolean columns
    if IsDeleteCoinc:
        if not IsFlagConflicts: df.drop(confl_col, axis=1, inplace=True)
        df.drop(coinc_col, axis=1, inplace=True)
        return df[df[kp_col]].drop(kp_col, axis=1)
    else:
        return df
    
def IsRowConflict(df, i, iPrev, col):
    if not IsNullCell(df, iPrev, col):
        if not IsNullCell(df, i, col): return True
    return False

#TRUE if row i of df col is NaN
def IsNullCell(df, i, col):
    if pd.isnull(df[col].iloc[i]): return True
    return False

#Returns the index of the previous row (not used)
def IndexPrev(df, idx):
    return df.index.values[df.index.get_loc(idx) - 1]
    

In [16]:
df = pd.read_csv('Row_rollup.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
timestamp    10 non-null datetime64[ns]
Its_on       4 non-null float64
Its_off      2 non-null float64
Other        4 non-null float64
Comment      10 non-null object
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 528.0+ bytes


In [17]:
df

,timestamp,Its_on,Its_off,Other,Comment
0,2020-01-08 02:00:00,1.0,NaN,NaN,Coincident - Should roll up
1,2020-01-08 02:00:00,NaN,NaN,300.0,Coincident - Should roll up
2,2020-01-08 03:00:00,NaN,1.0,NaN,Not coincident
3,2020-01-08 04:00:00,2.0,NaN,NaN,Conflict
4,2020-01-08 04:00:00,1.0,NaN,NaN,Conflict
5,2020-01-08 04:30:00,NaN,NaN,400.0,Not coincident
6,2020-01-08 16:00:00,NaN,NaN,100.0,Coincident - Should roll up
7,2020-01-08 16:00:00,NaN,3.0,NaN,Coincident - Should roll up
8,2020-01-08 16:00:00,1.0,NaN,NaN,Coincident - Should roll up
9,2020-01-08 20:00:00,NaN,NaN,800.0,Not coincident


In [18]:
lst_cols = ['Its_on','Its_off','Other']

Example 1:  Column values are not in conflict, so rows 0 and 1 get consolidated

In [19]:
RollupCoincidentRows(df.loc[0:2], 'timestamp', lst_cols, [], True, False)

,timestamp,Its_on,Its_off,Other,Comment,keep,RowConflict,IsCoincident
0,2020-01-08 02:00:00,1.0,NaN,300.0,Coincident - Should roll up,True,False,True
1,2020-01-08 02:00:00,NaN,NaN,300.0,Coincident - Should roll up,False,False,True
2,2020-01-08 03:00:00,NaN,1.0,NaN,Not coincident,True,False,False


In [20]:
RollupCoincidentRows(df.loc[0:2], 'timestamp', lst_cols, [], True, True)

,timestamp,Its_on,Its_off,Other,Comment,RowConflict
0,2020-01-08 02:00:00,1.0,NaN,300.0,Coincident - Should roll up,False
2,2020-01-08 03:00:00,NaN,1.0,NaN,Not coincident,False


Example 2:  Rows 3 and 4 are in conflict; Since lst_override is empty, rows do not get consolidated

In [21]:
RollupCoincidentRows(df.loc[3:4], 'timestamp', lst_cols, [], True, False)

,timestamp,Its_on,Its_off,Other,Comment,keep,RowConflict,IsCoincident
3,2020-01-08 04:00:00,2.0,NaN,NaN,Conflict,True,False,True
4,2020-01-08 04:00:00,1.0,NaN,NaN,Conflict,True,True,True


Example 3: Same data rows as Example 2, but `Its_on` column listed as ok to override; this causes rows to be consolidated and retains value from last coincident row."

In [22]:
RollupCoincidentRows(df.loc[3:4], 'timestamp', lst_cols, ['Its_on'], True, True)

,timestamp,Its_on,Its_off,Other,Comment,RowConflict
3,2020-01-08 04:00:00,1.0,NaN,NaN,Conflict,False


Example 4: Three coincident rows (6, 7 and 8 in original DataFrame)

In [23]:
RollupCoincidentRows(df.loc[6:8], 'timestamp', lst_cols, [], True, False)

,timestamp,Its_on,Its_off,Other,Comment,keep,RowConflict,IsCoincident
6,2020-01-08 16:00:00,1.0,3.0,100.0,Coincident - Should roll up,True,False,True
7,2020-01-08 16:00:00,NaN,3.0,NaN,Coincident - Should roll up,False,False,True
8,2020-01-08 16:00:00,1.0,NaN,NaN,Coincident - Should roll up,False,False,True


In [24]:
RollupCoincidentRows(df.loc[6:8], 'timestamp', lst_cols, [], True, True)

,timestamp,Its_on,Its_off,Other,Comment,RowConflict
6,2020-01-08 16:00:00,1.0,3.0,100.0,Coincident - Should roll up,False


Example 5: All Data - 6 rows [0,2,3,5,6,9] remain after consolidation

In [25]:
RollupCoincidentRows(df, 'timestamp', lst_cols, lst_cols, True, True)

,timestamp,Its_on,Its_off,Other,Comment,RowConflict
0,2020-01-08 02:00:00,1.0,NaN,300.0,Coincident - Should roll up,False
2,2020-01-08 03:00:00,NaN,1.0,NaN,Not coincident,False
3,2020-01-08 04:00:00,1.0,NaN,NaN,Conflict,False
5,2020-01-08 04:30:00,NaN,NaN,400.0,Not coincident,False
6,2020-01-08 16:00:00,1.0,3.0,100.0,Coincident - Should roll up,False
9,2020-01-08 20:00:00,NaN,NaN,800.0,Not coincident,False
